## Examples for using prunito

In [2]:
import prunito
prunito.__version__

'0.93.0'

### Searching and parsing UniProtKB data

Prunito allows searching uniprot.org and parsing entries in flat file (text) format. First, import the relevant package:

In [3]:
from prunito import uniprot as up

A simple search for all reviewed UniProtKB entries with 'tax' in their names.

In [4]:
result = up.search_reviewed('name:tax')

Check how many hits this search retrieved.

In [5]:
result.size()
# or
#len(result)

41

What would happen if a given query had many hits? By default, the maximum number of hits retrieved is 500. This can be changed using the parameter 'limit'.

Let's re-run the previous search but this time not just for reviewed (i.e. Swiss-Prot) entries but all of UniProtKB:

In [6]:
huge = up.search('name:tax')

ExcessiveDataError: Number of hits exceeds limit. Limit: 500. Actual hits: 1890. Please adjust limit or query.

The search function throws an ExcessiveDataError. Now either the limit of retrieved entries has to be increased or the error could be caught and handled gracefully.

In [7]:
# Approach 1: increase the limit of allowed hits
# new_limit = up.search('name:tax', limit=2000)
# new_limit.size()
#
# Approach 2: Handle the error and move on
from prunito.utils import ExcessiveDataError
try:
    new_result = up.search('name:tax')
except ExcessiveDataError:
    print('Too many hits! Try again.')

Too many hits! Try again.


Back to the initial result, the one with size of 41. What does 41 mean? UniProtKB entries. And these entries we would like to parse. As prunito provides both functionality for both searching and parsing UniProt, a search result is directly iterable for convenience:

In [8]:
entries = list(result)
# or
# for entry in result: ...

Iterate over the 41 entries, printing out primary accessions and recommended full names. Both fields are provided for convenience (unlike Biopython's Record object which does not have this).

In [9]:
for entry in entries:
    print(entry.primary_accession, entry.recommended_full_name)

Q06507 Cyclic AMP-dependent transcription factor ATF-4
P18848 Cyclic AMP-dependent transcription factor ATF-4
Q9Y6D9 Mitotic spindle assembly checkpoint protein MAD1
P47911 60S ribosomal protein L6
P10070 Zinc finger protein GLI2
Q0VGT2 Zinc finger protein GLI2
O14908 PDZ domain-containing protein GIPC1
Q02878 60S ribosomal protein L6
O14910 Protein lin-7 homolog A
Q10586 D site-binding protein
P22063 Contactin-2
Q02246 Contactin-2
Q61330 Contactin-2
P43489 Tumor necrosis factor receptor superfamily member 4
Q08AM6 Protein VAC14 homolog
P47030 Protein TAX4
Q5R4U3 Tax1-binding protein 1 homolog
Q4U0X7 Protein Tax-3
P0C222 Protein Tax-1
P23510 Tumor necrosis factor ligand superfamily member 4
A6ZPP1 Protein TAX4
Q2KJE0 Tax1-binding protein 1 homolog
Q86VP1 Tax1-binding protein 1
P14079 Protein Tax-1
O35426 X-box-binding protein 1
P17861 X-box-binding protein 1
Q9DBG9 Tax1-binding protein 3
Q6P132 Tax1-binding protein 1 homolog B
Q4QQV1 Tax1-binding protein 3
Q1LWB0 Tax1-binding protein 1

Which methods and fields are available on a Record object?

In [10]:
for item in dir(entries[0]):
    if not item.startswith('_'):
        print(item)

accessions
all_pubmed_ids
annotation_update
as_fasta
comments
created
cross_references
data_class
description
ec_numbers
entry_name
features
gene_name
host_organism
host_taxonomy_id
isoforms
keywords
molecule_type
organelle
organism
organism_classification
primary_accession
primary_gene_name
recommended_full_name
references
seqinfo
sequence
sequence_length
sequence_update
taxonomy_id


Get isoforms for those entries that have them. We use the presence of a keyword, 'Alternative splicing', as a filter here.

In [19]:
for e in entries:
    if 'Alternative splicing' in e.keywords:
        for i in e.isoforms():
            print(i)

>sp|Q9Y6D9-2|MD1L1_HUMAN Isoform 2 of Mitotic spindle assembly checkpoint protein MAD1 OS=Homo sapiens (Human). OX=['9606']
MLPARGCVRKRTVWPRLARVLIVTLLTLELSYAPLPCQLSGVPYNTGDPVGRWARPCIWP
CPWHTTINALKGRISELQWSVMDQEMRVKRLESEKQELQEQLDLQHKKCQEANQKIQELQ
ASQEARADHEQQIKDLEQKLSLQEQDAAIVKNMKSELVRLPRLERELKQLREESAHLREM
RETNGLLQEELEGLQRKLGRQEKMQETLVGLELENERLLAKLQSWERLDQTMGLSIRTPE
DLSRFVVELQQRELALKDKNSAVTSSARGLEKARQQLQEELRQVSGQLLEERKKRETHEA
LARRLQKRVLLLTKERDGMRAILGSYDSELTPAEYSPQLTRRMREAEDMVQKVHSHSAEM
EAQLSQALEELGGQKQRADMLEMELKMLKSQSSSAEQSFLFSREEADTLRLKVEELEGER
SRLEEEKRMLEAQLERRALQGDYDQSRTKVLHMSLNPTSVARQRLREDHSQLQAECERLR
GLLRAMERGGTVPADLEAAAASLPSSKEVAELKKQVESAELKNQRLKEVFQTKIQEFRKA
CYTLTGYQIDITTENQYRLTSLYAEHPGDCLIFKATSPSGSKMQLLETEFSHTVGELIEV
HLRRQDSIPAFLSSLTLELFSRQTVA
>sp|P10070-1|GLI2_HUMAN Isoform 1 of Zinc finger protein GLI2 OS=Homo sapiens (Human). OX=['9606']
MALTSINATPTQLSSSSNCLSDTNQNKQSSESAVSSTVNPVAIHKRSKVKTEPEGLRPAS
PLALTQGQVSGHGSCGCALPLSQEQLADLKEDLDRDDCKQEAEVVIYETNCHWEDCTKEY
DTQEQLVHHINNEHIHGE

We would like to run a FASTA similarity search against Swiss-Prot for one of the sequences. Let's take the canonical sequence of the first entry in entries.

Here we use the ebiwebservices module from prunito.

In [11]:
from prunito import ebiwebservices as ews

In [12]:
first_entry = entries[0]
print(first_entry.as_fasta())

>sp|Q06507|ATF4_MOUSE Cyclic AMP-dependent transcription factor ATF-4 OS=Mus musculus (Mouse). OX=['10090']
MTEMSFLNSEVLAGDLMSPFDQSGLGAEESLGLLDDYLEVAKHLKPHGFSSDKAGSSEWP
AMDDGLASASDTGKEDAFSGTDWMLEKMDLKEFDFDALFRMDDLETMPDELLTTLDDTCD
LFAPLVQETNKEPPQTVNPIGHLPESLIKVDQVAPFTFLQPFPCSPGVLSSTPEHSFSLE
LGSEVDISEGDRKPDSAAYITLIPPCVKEEDTPSDNDSGICMSPESYLGSPQHSPSTSRA
PPDNLPSPGGSRGSPRPKPYDPPGVSLTAKVKTEKLDKKLKKMEQNKTAATRYRQKKRAE
QEALTGECKELEKKNEALKEKADSLAKEIQYLKDLIEEVRKARGKKRVP


In [13]:
# fasta_search() defaults to Swiss-Prot as target set
similar = ews.fasta_search(first_entry.as_fasta())

No email address for EBI web service set.
This is required for the services to work.
Use set_email() to provide an email address.


Aha, we get a message that EBI web services require an email address. 

In [14]:
ews.set_email('some@gmx.de')

In [15]:
similar = ews.fasta_search(first_entry.as_fasta())

In [16]:
print(similar.text[:800])

# /nfs/public/release/wp-jdispatcher/latest/appbin/linux-x86_64/fasta-36.3.7b/fasta36 -l /nfs/public/ro/es/data/idata/latest/fastacfg/fasta3db -L -T 8 -p -m "F9 fasta-R20180430-165854-0786-71182654-p1m.m9" @:1- +uniprotkb_swissprot+
FASTA searches a protein or DNA sequence data bank
 version 36.3.7b Jun, 2015(preload9)
Please cite:
 W.R. Pearson & D.J. Lipman PNAS (1988) 85:2444-2448

Query: @
  1>>>sp|Q06507|ATF4_MOUSE Cyclic AMP-dependent transcription factor ATF-4 OS=Mus musculus (Mouse). OX=['10090'] - 349 aa
Library: UniProtKB/Swiss-Prot
  199856860 residues in 557275 sequences

Statistics:  Expectation_n fit: rho(ln(x))= 9.2785+/-0.000157; mu= 0.6906+/- 0.009
 mean_var=128.4971+/-24.729, 0's: 395 Z-trim(117.9): 509  B-trim: 0 in 0/66
 Lambda= 0.113143
 statistics sampled from 60000 (


Hiow about using InterPro's HMMER search instead of FASTA?

In [59]:
from prunito import interpro

In [ ]:
#ip_similar = interpro.search_phmmer(first_entry.as_fasta())

Do some of the 41 entries contain the same PubMed IDs? Let's find the 5 most common ones.

In [23]:
from collections import Counter

In [22]:
c = Counter()
for e in entries:
    c.update(e.all_pubmed_ids)
print(c.most_common(5))

[('15489334', 24), ('20068231', 9), ('14702039', 8), ('23186163', 8), ('21269460', 7)]


Which are the accession numbers and species of those 24 entries containing the most common one (15489334)?

In [24]:
for e in entries:
    if '15489334' in e.all_pubmed_ids:
        print(e.primary_accession, e.organism)

Q06507 Mus musculus (Mouse).
P18848 Homo sapiens (Human).
Q9Y6D9 Homo sapiens (Human).
P47911 Mus musculus (Mouse).
Q0VGT2 Mus musculus (Mouse).
O14908 Homo sapiens (Human).
Q02878 Homo sapiens (Human).
O14910 Homo sapiens (Human).
Q10586 Homo sapiens (Human).
Q61330 Mus musculus (Mouse).
P43489 Homo sapiens (Human).
Q08AM6 Homo sapiens (Human).
P23510 Homo sapiens (Human).
Q86VP1 Homo sapiens (Human).
O35426 Mus musculus (Mouse).
P17861 Homo sapiens (Human).
Q9DBG9 Mus musculus (Mouse).
Q4QQV1 Rattus norvegicus (Rat).
O14907 Homo sapiens (Human).
Q66HA4 Rattus norvegicus (Rat).
Q9UQ35 Homo sapiens (Human).
Q13884 Homo sapiens (Human).
Q3UKC1 Mus musculus (Mouse).
Q9NPB6 Homo sapiens (Human).


So, which paper is hiding behind this PMID 15489334? Here we use another module from prunito.

In [25]:
from prunito import europepmc as epmc

In [26]:
paper = epmc.get_pmid_metadata('15489334')

EuropePMC returns data for example in JSON format. Currently, there are no convenience methods to access the fields therein.

In [29]:
paper.json()['resultList']['result'][0]['title']

'The status, quality, and expansion of the NIH full-length cDNA project: the Mammalian Gene Collection (MGC).'

In [32]:
paper.json()['resultList']['result'][0]['abstractText']

"The National Institutes of Health's Mammalian Gene Collection (MGC) project was designed to generate and sequence a publicly accessible cDNA resource containing a complete open reading frame (ORF) for every human and mouse gene. The project initially used a random strategy to select clones from a large number of cDNA libraries from diverse tissues. Candidate clones were chosen based on 5'-EST sequences, and then fully sequenced to high accuracy and analyzed by algorithms developed for this project. Currently, more than 11,000 human and 10,000 mouse genes are represented in MGC by at least one clone with a full ORF. The random selection approach is now reaching a saturation point, and a transition to protocols targeted at the missing transcripts is now required to complete the mouse and human collections. Comparison of the sequence of the MGC clones to reference genome sequences reveals that most cDNA clones are of very high sequence quality, although it is likely that some cDNAs may c

As the paper mentions the Mammalian Gene Collection, why not search EuropePMC for articles mentioning the collection in their abstracts?

In [33]:
mgc_papers = epmc.search('abstract:"Mammalian Gene Collection"')

In [41]:
mgc_papers.json()['hitCount']

24

In [44]:
for idx, hit in enumerate(mgc_papers.json()['resultList']['result']):
    print(idx, hit['title'])

0 Identification of candidate transcription factor binding sites in the cattle genome.
1 Selenoproteins in bladder cancer.
2 NSrp70 is a novel nuclear speckle-related protein that modulates alternative pre-mRNA splicing in vivo.
3 Generation of a genome scale lentiviral vector library for EF1α promoter-driven expression of human ORFs and identification of human genes affecting viral titer.
4 The completion of the Mammalian Gene Collection (MGC).
5 A high-throughput platform for lentiviral overexpression screening of the human ORFeome.
6 PRFdb: a database of computationally predicted eukaryotic programmed -1 ribosomal frameshift signals.
7 Transcriptome analysis of a cDNA library from adult human epididymis.
8 Construction and characterization of a normalized yeast two-hybrid library derived from a human protein-coding clone collection.
9 Evaluation of vector-primed cDNA library production from microgram quantities of total RNA.
10 Concatenation cDNA sequencing for transcriptome analysi

Each hit/paper has many extra data fields including DOI, PubMed ID etc. If the abstract is needed, resulttype='core' has to be specified as a search parameter.

In [50]:
for k, v in mgc_papers.json()['resultList']['result'][0].items():
    print(k + ':\t' + str(v))

id:	23433959
source:	MED
pmid:	23433959
pmcid:	PMC4357788
doi:	10.1016/j.gpb.2012.10.004
title:	Identification of candidate transcription factor binding sites in the cattle genome.
authorString:	Bickhart DM, Liu GE.
journalTitle:	Genomics Proteomics Bioinformatics
issue:	3
journalVolume:	11
pubYear:	2013
journalIssn:	1672-0229; 2210-3244; 
pageInfo:	195-198
pubType:	research support, non-u.s. gov't; research-article; research support, u.s. gov't, non-p.h.s.; journal article; 
isOpenAccess:	Y
inEPMC:	Y
inPMC:	N
hasPDF:	Y
hasBook:	N
hasSuppl:	Y
citedByCount:	4
hasReferences:	Y
hasTextMinedTerms:	Y
hasDbCrossReferences:	N
hasLabsLinks:	Y
hasTMAccessionNumbers:	N
firstPublicationDate:	2013-02-01
